Consider vd, a dictionary of dictionaries of viral sequence information, with this structure:

`vd = {
"Seq1": {"Host": "human","Country": "Italy", "CollectionDate": "2022-03-01", "ListOfMutations": "A,B,C,D,E", },
"Seq2": {"Host": "human", "Country": "Italy", "CollectionDate": "2023-03-01", "ListOfMutations": "A,B,C,D"},
"Seq3": {"Host": "human", "Country": "France", "CollectionDate": "2022-03-01", "ListOfMutations": "A,B,C,E,X,Y,Z"},
"Seq4": {"Host": "mouse", "Country": "Italy", "CollectionDate": "2023-04-01", "ListOfMutations": "A,D,E"}}`

### 1) [5pts]
Write a Python code that computes and prints the all the distinct mutations (i.e., letters) that are present in at least one sequence of vd. Note
that the value corresponding to the ListOfMutations key in the dictionary is provided as a string.

In [102]:
import time
from curses.ascii import isdigit

vd = {"Seq1": {"Host": "human","Country": "Italy", "CollectionDate": "2022-03-01", "ListOfMutations": "A,B,C,D,E", },
"Seq2": {"Host": "human", "Country": "Italy", "CollectionDate": "2023-03-01", "ListOfMutations": "A,B,C,D"},
"Seq3": {"Host": "human", "Country": "France", "CollectionDate": "2022-03-01", "ListOfMutations": "A,B,C,E,X,Y,Z"},
"Seq4": {"Host": "mouse", "Country": "Italy", "CollectionDate": "2023-04-01", "ListOfMutations": "A,D,E"}}

new=','.join([vl['ListOfMutations'] for ky,vl in vd.items()])
new=new.split(',')
print(set(new))

# Or the other way

total_list= [vl["ListOfMutations"].split(',')for ky,vl in vd.items()]
total_list=[''.join(lis) for lis in total_list]
total_list=''.join(total_list)
distinct_mutation=set(total_list)
print(distinct_mutation)


#Or GPT suggested
mutations = {m for v in vd.values() for m in v['ListOfMutations'].split(',')}
print(mutations)



{'B', 'X', 'E', 'Y', 'C', 'D', 'A', 'Z'}
{'B', 'X', 'E', 'Y', 'C', 'D', 'A', 'Z'}
{'B', 'X', 'E', 'Y', 'C', 'D', 'A', 'Z'}


### 2) [4pts]
 Consider a file where viral lineages are defined in terms of lists of mutations. The file lineage_characterization.tsv has the following structure:
 each line has the name of a lineage, separated with a tab from a comma-separated list of mutations. Write a Python code that parses the file and
 stores its content into a dictionary lc, where a key is a lineage (e.g., ‘L4’) and a value is a list of mutations (e.g., [‘B’,‘C’,‘D’]).
  Example:
`L4 B,C,D
L2 B,C,D,E
L1 A,B,C
L3 G,H,J`

In [98]:

with open('lineage_characterization.tsv', 'w') as f:
    content= f'L4\tB,C,D\nL2\tB,C,D,E\nL1\tA,B,C\nL3\tG,H,J\n'
    f.write(content)

with open('lineage_characterization.tsv', 'r') as f:
     list_pair=[line.strip().split('\t') for line in f]
     lc={line[0]: line[1].split(',') for line in list_pair}
print(lc)




{'L4': ['B', 'C', 'D'], 'L2': ['B', 'C', 'D', 'E'], 'L1': ['A', 'B', 'C'], 'L3': ['G', 'H', 'J']}


### 3) [7pts]
 Implement the assignToLineage function, which takes in input a list of mutations and returns the lineage (stored in the dictionary lc created in Ex.
 that is closest to the input list. The lineages with the minimum distance must be returned. In case of ties, it is enough to return one lineage among the possible ones.
 Example: The input sequence has mutations A,B,C,D

- Its distance with L1 is 1: from ABCD to ABC, we lose one element (+1)
-
- Its distance with L2 is 2: from ABCD to BCDE we lost A (+1) and added E (+1)
-
- Its distance with L3 is 7 (from ABCD to GHJ we lost four letters (+4) and added three new ones (+3)

In this case, the returned lineage is L1 (because it has distance 1 from ABCD and 1 is the minimum among 1, 2 and 7).

Hint 1: The distance can be conveniently computed using set-oriented functions.

Hint 2: Auxiliary functions (as usual) may be used. It may be useful to implement an auxiliary function distance that returns an integer value, called within assignToLineage.

In [52]:
#Or define this function and use it or directly
def dist(ls1,ls2):
       set1=set(ls1)
       set2=set(ls2)
       return len(set1.symmetric_difference(set2))


def assignToLineage(mut:list):
    #mut=set(mut)
    #len(set(ref).symmetric_difference(mut))
    distance={lin: dist(mut,ref) for lin,ref in lc.items()}
    min_distance=min(distance.values())
    for lin,dis in distance.items():
        if dis==min_distance:
            return lin


assignToLineage(['A','B','C','D'])



'L4'

### 4) [3pts]
 Add to each sequence of the vd dictionary a key-value pair that stores the “Lineage”. Its value is the closest Lineage possible, as calculated by the
 assignToLineage function.

In [103]:
import copy
vd_v= copy.deepcopy(vd)
for vl in vd_v.values():
    vl['Lineage']=assignToLineage(vl['ListOfMutations'].split(','))

### 6) [7pts]
Sketch the basic infrastructure to handle this scenario with classes: Sequences are represented by a class. A sequence has a sequence_id, a collection_date,
 and a list of mutations. Directly in the constructor insert a syntactic check on the date format, which should be ‘yyyy-mm-dd’, else an appropriate
 exception is thrown.
Datasets contain a set of sequences and a counter of original_sequences contained in the set (i.e., those that have up to 2 mutations). They have a method
o insert a new sequence, that checks that there does not exist a sequence with the same id (else, it raises an exception) and updates the original_sequences
 counter.

In [136]:
class NoDouplicate(Exception):
    def __init__(self, msg):
        self.msg = msg
    def __str__(self):
        return self.msg

class Sequence:
    def __init__(self,id,date,mutlist):
        # check the other versions down below
        check=[]
        for part in [date[:4],date[5:7],date[8:]]:
            if part.isdigit():
                check.append(part)
        if '-'.join(check)== date:
            self.collection_date=date
        else:
                raise ValueError ('Date must be in yyyy-mm-dd format')
        self.id=id
        self.mutlist=mutlist

    def __iter__(self):
        return iter(self.mutlist)
    def __str__(self):
        return self.id


class Datasets:
    def __init__(self):
        self.set=set([])
        self.original_sequences=0

    def addSequence(self,seq:Sequence):
        if seq.id not in [s.id for s in self.set]:
            self.set.add(seq)
            if len(seq.mutlist) <=2:
                self.original_sequences +=1
        else:
            raise NoDouplicate ('the inserted sequence is already in the set')




In [ ]:
# GPT recommended version
from datetime import datetime
try:
    datetime.strptime(date, '%y-%m-%d')
except ValueError as e:
    print(e)
self.collection_date=date

In [ ]:
#The other version that's compact
if len(date) == 10 and date[4] == '-' and date[7] == '-' and all(part.isdigit() for part in [date[:4], date[5:7], date[8:]]):
    self.collection_date = date
else:
    raise FormatError('Date must be in yyyy-mm-dd format')

